In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst.summit.utils.tmaUtils import TMAEventMaker, plotEvent
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst_efd_client import EfdClient

In [ ]:
dayObs = 20231123
client = EfdClient('usdf_efd')
eventMaker = TMAEventMaker(client=client)
events = eventMaker.getEvents(dayObs)



In [ ]:
events[351].type.name

In [ ]:
seqNum = 351
start = events[seqNum].begin + TimeDelta(1.0, format='sec')
end = events[seqNum].end - TimeDelta(1.0, format='sec')
print(start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition','demandPosition', 'timestamp'],  start, end)    

az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
az_demand = np.array(az.values[:,1])
el_demand = np.array(el.values[:,1])

times_az = az.values[:,2]
times_el = el.values[:,2]
# The fits are much better if the time variable
# is centered in the interval
fit_times_az = [(times_az[i]-times_az[int(len(az.values) / 2)]) for i in range(len(times_az))]
fit_times_el = [(times_el[i]-times_el[int(len(el.values) / 2)]) for i in range(len(times_el))]

# Fit with a polynomial
az_fit = np.polyfit(fit_times_az, az_vals, 4)
el_fit = np.polyfit(fit_times_el, el_vals, 4)
az_model = np.polyval(az_fit, fit_times_az)
el_model = np.polyval(el_fit, fit_times_el)

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600
az_error_demand = (az_vals - az_demand) * 3600
el_error_demand = (el_vals - el_demand) * 3600
# Calculate RMS
az_rms = np.sqrt(np.mean(az_error * az_error))
el_rms = np.sqrt(np.mean(el_error * el_error))

# Calculate Image impact RMS
# We are less sensitive to Az errors near the zenith
image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
image_el_rms = el_rms 

fig = plt.figure(figsize = (8,12))
plt.subplots_adjust(wspace=0.3, hspace=0.3)
plt.suptitle(f"MT Mount Jitter - {dayObs} - {seqNum}", fontsize = 18)
plt.subplot(3,2,1)
ax1 = az['actualPosition'].plot(legend=True, color='red')
ax1.set_title("Azimuth")
ax1.set_ylabel("Degrees")
plt.subplot(3,2,2)
ax3 = el['actualPosition'].plot(legend=True, color='green')
ax3.set_title("Elevation")
ax3.set_ylabel("Degrees")

plt.subplot(3,2,3)
plt.plot(fit_times_az, az_error, color='red')
plt.title(f"Azimuth RMS error: Poly fit", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.subplot(3,2,4)
plt.plot(fit_times_el, el_error, color='green')
plt.title(f"Elevation RMS error: Poly fit", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")

plt.subplot(3,2,5)
plt.plot(fit_times_az, az_error_demand, color='red')
plt.title(f"Azimuth RMS error: Actual - Demand", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.subplot(3,2,6)
plt.plot(fit_times_el, el_error_demand, color='green')
plt.title(f"Elevation RMS error: Actual - Demand", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
#plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Mount_plot_{dayObs}_{seqNum}.png")

In [ ]:
fig = plt.figure(figsize = (12,6))
az_times = az.values[:,2]- az.values[0,2]
plt.subplot(1,2,1)
plt.title(f"Az - actual vs demand - {dayObs} - {seqNum}", fontsize = 18)
plt.plot(az_times, az.values[:,0], label='actual', marker='x')
plt.plot(az_times, az.values[:,1], color='red', ls = '--', label='demand', marker='+')
plt.legend()
plt.xlim(40.75, 41.5)
plt.ylim(87.7475, 87.751)

el_times = el.values[:,2]- el.values[0,2]
plt.subplot(1,2,2)
plt.title(f"El - actual vs demand - {dayObs} - {seqNum}", fontsize = 18)
plt.plot(el_times, el.values[:,0], label='actual', marker='x')
plt.plot(el_times, el.values[:,1], color='red', ls = '--', label='demand', marker='+')
plt.legend()
plt.xlim(16.4, 17.3)
plt.ylim(60.527, 60.530)
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Demand_glitches_{dayObs}_{seqNum}.png")

In [ ]:
seqNum = 351
start = events[seqNum].begin - TimeDelta(12.0, format='sec')
end = events[seqNum].end - TimeDelta(47.0, format='sec')
print(start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition','demandPosition', 'timestamp'],  start, end)    

ptg = await client.select_time_series('lsst.sal.MTPtg.currentTargetStatus', \
                                            ['demandAz', 'demandEl', 'timestamp'],  start, end)

ptg_times = ptg.values[:,2]# - ptg.values[0,2]
az_times = az.values[:,2]#- az.values[0,2]
el_times = el.values[:,2]#- el.values[0,2]

slew_start = events[seqNum-1].begin.unix_tai - 0.55# - az.values[0,2]
slew_end = events[seqNum-1].end.unix_tai

fig = plt.figure(figsize = (12,6))
plt.suptitle(f"Slew Plot - {dayObs} - {seqNum}", fontsize = 18)
plt.subplot(1,2,1)
plt.title(f"Az - actual vs demand", fontsize = 18)
plt.plot(az_times, az.values[:,0], label='actual')
plt.plot(ptg_times, ptg.values[:,0], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [87.35, 88.0], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [87.35, 88.0], ls='--', color='green', label='End Slew - In Position')
plt.xlabel("Time(sec)")
plt.legend()

plt.subplot(1,2,2)
plt.title(f"El - actual vs demand", fontsize = 18)
plt.plot(el_times, el.values[:,0], label='actual')
plt.plot(ptg_times, ptg.values[:,1], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [60.0, 63.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [60.0, 63.5], ls='--', color='green', label='End Slew - In Position')
plt.xlabel("Time(sec)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Slew_Plot_{dayObs}_{seqNum}.png")

# Get this working!!!
## Need to interpolate the ptg to match the az and el.

In [ ]:
seqNum = 351
start = events[seqNum].begin + TimeDelta(1.0, format='sec')
end = events[seqNum].end - TimeDelta(1.0, format='sec')
print(start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition','demandPosition', 'timestamp'],  start, end)    

ptg = await client.select_time_series('lsst.sal.MTPtg.currentTargetStatus', \
                                            ['demandAz', 'demandEl', 'timestamp'],  \
                                      start - TimeDelta(1.0, format='sec'), end + TimeDelta(1.0, format='sec'))

ptg_times = ptg.values[:,2] + 0.05
az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
az_demand = np.array(ptg.values[:,0])
el_demand = np.array(ptg.values[:,1])


times_az = az.values[:,2]
times_el = el.values[:,2]

az_demand_interp = np.interp(times_az, ptg_times, az_demand)
el_demand_interp = np.interp(times_el, ptg_times, el_demand)

# The fits are much better if the time variable
# is centered in the interval
fit_times_az = [(times_az[i]-times_az[int(len(az.values) / 2)]) for i in range(len(times_az))]
fit_times_el = [(times_el[i]-times_el[int(len(el.values) / 2)]) for i in range(len(times_el))]

# Fit with a polynomial
az_fit = np.polyfit(fit_times_az, az_vals, 4)
el_fit = np.polyfit(fit_times_el, el_vals, 4)
az_model = np.polyval(az_fit, fit_times_az)
el_model = np.polyval(el_fit, fit_times_el)

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600
az_error_demand = (az_vals - az_demand_interp) * 3600
el_error_demand = (el_vals - el_demand_interp) * 3600
# Calculate RMS
az_rms = np.sqrt(np.mean(az_error * az_error))
el_rms = np.sqrt(np.mean(el_error * el_error))

# Calculate Image impact RMS
# We are less sensitive to Az errors near the zenith
image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
image_el_rms = el_rms 

fig = plt.figure(figsize = (8,12))
plt.subplots_adjust(wspace=0.3, hspace=0.3)
plt.suptitle(f"MT Mount Jitter - {dayObs} - {seqNum}", fontsize = 18)
plt.subplot(3,2,1)
ax1 = az['actualPosition'].plot(legend=True, color='red')
ax1.set_title("Azimuth")
ax1.set_ylabel("Degrees")
plt.subplot(3,2,2)
ax3 = el['actualPosition'].plot(legend=True, color='green')
ax3.set_title("Elevation")
ax3.set_ylabel("Degrees")

plt.subplot(3,2,3)
plt.plot(fit_times_az, az_error, color='red')
plt.title(f"Azimuth RMS error: Poly fit", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.subplot(3,2,4)
plt.plot(fit_times_el, el_error, color='green')
plt.title(f"Elevation RMS error: Poly fit", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")

plt.subplot(3,2,5)
plt.plot(fit_times_az, az_error_demand, color='red')
plt.title(f"Azimuth RMS error: Actual - Demand", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.subplot(3,2,6)
plt.plot(fit_times_el, el_error_demand, color='green')
plt.title(f"Elevation RMS error: Actual - Demand", fontsize=10)
plt.ylim(-1.0,1.0)
plt.xticks([])
plt.ylabel("ArcSeconds")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Mount_plot_MTPtg_Offset_0p05_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
seqNum = 351
start = events[seqNum].begin - TimeDelta(12.0, format='sec')
end = events[seqNum].end - TimeDelta(46.0, format='sec')
print(start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition','demandPosition', 'timestamp'],  start, end)    

ptg = await client.select_time_series('lsst.sal.MTPtg.currentTargetStatus', \
                                            ['demandAz', 'demandEl', 'timestamp'],  start, end)

ptg_times = ptg.values[:,2] + 0.05# - ptg.values[0,2]
az_times = az.values[:,2]#- az.values[0,2]
el_times = el.values[:,2]#- el.values[0,2]

az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
az_demand = np.array(ptg.values[:,0])
el_demand = np.array(ptg.values[:,1])

az_demand_interp = np.interp(az_times, ptg_times, az_demand)
el_demand_interp = np.interp(el_times, ptg_times, el_demand)

az_error_demand = (az_vals - az_demand_interp) * 3600
el_error_demand = (el_vals - el_demand_interp) * 3600

slew_start = events[seqNum-1].begin.unix_tai - 0.55#
slew_end = events[seqNum-1].end.unix_tai + 1.0

fig = plt.figure(figsize = (12,6))
plt.suptitle(f"Slew Plot - {dayObs} - {seqNum}", fontsize = 18)
plt.subplot(2,2,1)
plt.title(f"Az - actual vs demand", fontsize = 12)
plt.plot(az_times, az.values[:,0], label='actual')
plt.plot(ptg_times, ptg.values[:,0], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [87.35, 88.0], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [87.35, 88.0], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,2)
plt.title(f"El - actual vs demand", fontsize = 12)
plt.plot(el_times, el.values[:,0], label='actual')
plt.plot(ptg_times, ptg.values[:,1], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [60.0, 63.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [60.0, 63.5], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,3)
plt.plot(az_times, az_error_demand, color='blue')
plt.title(f"Azimuth RMS error: Actual - Demand", fontsize=12)
plt.ylim(-1.0,1.0)
plt.xlabel("Time(sec)")
plt.plot([slew_start, slew_start], [-0.5, 0.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')

plt.ylabel("ArcSeconds")

plt.subplot(2,2,4)
plt.plot(el_times, el_error_demand, color='blue')
plt.title(f"Elevation RMS error: Actual - Demand", fontsize=12)
plt.ylim(-1.0,1.0)

plt.plot([slew_start, slew_start], [-0.5, 0.55], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')
plt.xlabel("Time(sec)")
plt.ylabel("ArcSeconds")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Slew_Plot_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
seqNum = 351
start = events[seqNum].begin - TimeDelta(12.0, format='sec')
end = events[seqNum].end - TimeDelta(46.0, format='sec')
print(start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition','demandPosition', 'timestamp'],  start, end)    

ptg = await client.select_time_series('lsst.sal.MTPtg.currentTargetStatus', \
                                            ['demandAz', 'demandEl', 'timestamp'],  start, end)

ptg_times = ptg.values[:,2] + 0.05# - ptg.values[0,2]
az_times = az.values[:,2]#- az.values[0,2]
el_times = el.values[:,2]#- el.values[0,2]

az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
az_demand = np.array(ptg.values[:,0])
el_demand = np.array(ptg.values[:,1])

az_demand_interp = np.interp(az_times, ptg_times, az_demand)
el_demand_interp = np.interp(el_times, ptg_times, el_demand)

az_error_demand = (az_vals - az_demand_interp) * 3600
el_error_demand = (el_vals - el_demand_interp) * 3600

slew_start = events[seqNum-1].begin.unix_tai - 0.55#
slew_end = events[seqNum-1].end.unix_tai + 1.0

fig = plt.figure(figsize = (12,6))
plt.suptitle(f"Slew Plot - {dayObs} - {seqNum}", fontsize = 18)
plt.subplot(2,2,1)
plt.title(f"Az - actual vs demand", fontsize = 12)
plt.plot(az_times, az.values[:,0], label='actual')
plt.plot(ptg_times, ptg.values[:,0], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [87.35, 88.0], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [87.35, 88.0], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,2)
plt.title(f"El - actual vs demand", fontsize = 12)
plt.plot(el_times, el.values[:,0], label='actual')
plt.plot(ptg_times, ptg.values[:,1], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [60.0, 63.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [60.0, 63.5], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,3)
plt.plot(az_times, az_error_demand, color='blue')
plt.title(f"Azimuth RMS error: Actual - Demand", fontsize=12)
plt.ylim(-0.1, 0.1)
plt.xlabel("Time(sec)")
plt.plot([slew_start, slew_start], [-0.5, 0.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')
plt.plot([az_times[0], az_times[-1]], [-0.01, -0.01], color='black', ls='-.')
plt.plot([az_times[0], az_times[-1]], [0.01, 0.01], color='black', ls='-.')

plt.ylabel("ArcSeconds")

plt.subplot(2,2,4)
plt.plot(el_times, el_error_demand, color='blue')
plt.title(f"Elevation RMS error: Actual - Demand", fontsize=12)
plt.ylim(-0.1, 0.1)

plt.plot([slew_start, slew_start], [-0.5, 0.55], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')
plt.plot([el_times[0], el_times[-1]], [-0.01, -0.01], color='black', ls='-.')
plt.plot([el_times[0], el_times[-1]], [0.01, 0.01], color='black', ls='-.')

plt.xlabel("Time(sec)")
plt.ylabel("ArcSeconds")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Slew_Plot_BlowUp_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
seqNum = 351
start = events[seqNum].begin - TimeDelta(12.0, format='sec')
end = events[seqNum].end - TimeDelta(35.0, format='sec')
print(start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition','demandPosition', 'timestamp'],  start, end)    



az_times = az.values[:,2]- az.values[0,2]
el_times = el.values[:,2]- el.values[0,2]

az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
az_demand = np.array(az.values[:,1])
el_demand = np.array(el.values[:,1])

az_error_demand = (az_vals - az_demand) * 3600
el_error_demand = (el_vals - el_demand) * 3600

slew_start = events[seqNum-1].begin.unix_tai - az.values[0,2] - 0.55
slew_end = events[seqNum-1].end.unix_tai - az.values[0,2]# + 1.0

fig = plt.figure(figsize = (12,6))
plt.suptitle(f"Slew Plot - {dayObs} - {seqNum}", fontsize = 18)
plt.subplot(2,2,1)
plt.title(f"Az - actual vs demand", fontsize = 12)
plt.plot(az_times, az.values[:,0], label='actual')
plt.plot(az_times, az.values[:,1], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [87.35, 88.0], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [87.35, 88.0], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,2)
plt.title(f"El - actual vs demand", fontsize = 12)
plt.plot(el_times, el.values[:,0], label='actual')
plt.plot(el_times, el.values[:,1], color='red', ls = '--', label='demand')
plt.plot([slew_start, slew_start], [60.0, 63.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [60.0, 63.5], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,3)
plt.plot(az_times, az_error_demand, color='blue')
plt.title(f"Azimuth RMS error: Actual - Demand", fontsize=12)
plt.ylim(-0.05, 0.05)
plt.xlabel("Time(sec)")
plt.plot([slew_start, slew_start], [-0.5, 0.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')
plt.plot([az_times[0], az_times[-1]], [-0.01, -0.01], color='black', ls='-.')
plt.plot([az_times[0], az_times[-1]], [0.01, 0.01], color='black', ls='-.')

plt.ylabel("ArcSeconds")

plt.subplot(2,2,4)
plt.plot(el_times, el_error_demand, color='blue')
plt.title(f"Elevation RMS error: Actual - Demand", fontsize=12)
plt.ylim(-0.05, 0.05)

plt.plot([slew_start, slew_start], [-0.5, 0.55], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')
plt.plot([el_times[0], el_times[-1]], [-0.01, -0.01], color='black', ls='-.')
plt.plot([el_times[0], el_times[-1]], [0.01, 0.01], color='black', ls='-.')

plt.xlabel("Time(sec)")
plt.ylabel("ArcSeconds")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Slew_Plot_BlowUp_MTMount_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
seqNum = 431
start = events[seqNum-1].begin + TimeDelta(40.0, format='sec')
end = events[seqNum].end - TimeDelta(35.0, format='sec')
print(start, end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition','demandPosition', 'timestamp'],  start, end)    



az_times = az.values[:,2]- az.values[0,2]
el_times = el.values[:,2]- el.values[0,2]

az_vals = np.array(az.values[:,0])
el_vals = np.array(el.values[:,0])
az_demand = np.array(az.values[:,1])
el_demand = np.array(el.values[:,1])

az_error_demand = (az_vals - az_demand) * 3600
el_error_demand = (el_vals - el_demand) * 3600

slew_start = events[seqNum-1].begin.unix_tai - az.values[0,2] - 0.55
slew_end = events[seqNum-1].end.unix_tai - az.values[0,2]# + 1.0

fig = plt.figure(figsize = (12,6))
plt.suptitle(f"Slew Plot - {dayObs} - {seqNum}", fontsize = 18)
plt.subplot(2,2,1)
plt.title(f"Az - actual vs demand", fontsize = 12)
plt.plot(az_times, az.values[:,0], label='actual')
plt.plot(az_times, az.values[:,1], color='red', ls = '--', label='demand')
#plt.plot([slew_start, slew_start], [87.35, 88.0], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [87.35, 88.0], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,2)
plt.title(f"El - actual vs demand", fontsize = 12)
plt.plot(el_times, el.values[:,0], label='actual')
plt.plot(el_times, el.values[:,1], color='red', ls = '--', label='demand')
#plt.plot([slew_start, slew_start], [60.0, 63.5], ls='--', color='black', label='Start Slew')
#plt.plot([slew_end, slew_end], [60.0, 63.5], ls='--', color='green', label='End Slew - In Position')
plt.xticks([])

plt.legend()

plt.subplot(2,2,3)
plt.plot(az_times, az_error_demand, color='blue')
plt.title(f"Azimuth RMS error: Actual - Demand", fontsize=12)
#plt.ylim(-10, 10)
plt.xlabel("Time(sec)")
#plt.plot([slew_start, slew_start], [-0.5, 0.5], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')
plt.plot([az_times[0], az_times[-1]], [-0.01, -0.01], color='black', ls='-.')
plt.plot([az_times[0], az_times[-1]], [0.01, 0.01], color='black', ls='-.')

plt.ylabel("ArcSeconds")

plt.subplot(2,2,4)
plt.plot(el_times, el_error_demand, color='blue')
plt.title(f"Elevation RMS error: Actual - Demand", fontsize=12)
plt.ylim(-0.05, 0.05)

plt.plot([slew_start, slew_start], [-0.5, 0.55], ls='--', color='black', label='Start Slew')
plt.plot([slew_end, slew_end], [-0.5, 0.5], ls='--', color='green', label='End Slew - In Position')
plt.plot([el_times[0], el_times[-1]], [-0.01, -0.01], color='black', ls='-.')
plt.plot([el_times[0], el_times[-1]], [0.01, 0.01], color='black', ls='-.')

plt.xlabel("Time(sec)")
plt.ylabel("ArcSeconds")

plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Slew_Plot_BlowUp_MTMount_{dayObs}_{seqNum}.png")